In [1]:
# Pips
!pip install pandas
!pip install scikit-learn
!pip install lightgbm
!pip install xgboost
!pip install catboost

In [2]:
# Function / Package Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import accuracy_score
from scipy.stats import friedmanchisquare
import scikit_posthocs as sp
from sklearn.ensemble import VotingClassifier, VotingRegressor
import warnings; warnings.filterwarnings('ignore')

In [3]:
# Data Imports / Manipulation
df1 = pd.read_csv(r"Representative_kinases (1)\Representative_kinases\Rep_kinases_dataset.csv", sep= "	")
df2 = pd.read_csv(r"Dark Matter (I'm not running descriptors again).csv")

# Sort Kinases, Merge, etc.
# end with df_Kinase / df_KinaseGroup (~x20)

In [4]:
# Define Functions
def clean_df1(df1):
    df1 = df1[df1["p_standard_type"] == "pIC50"]
    df1["SMILES"] = df1["NonstereoAromaticSMILES"]
    df1["Value"] = df1["p_value"]
    df1["Class"] = 1
    df1["Kinase"] = df1["Kinase_name"]
    df1["Group"] = df1["Kinase_group"]    
    dfx = df1[["SMILES", "Class", "Value", "Kinase", "Group"]]
    del df1
    
    return dfx

def clean_df2(df2):
    df2["Value"] = 0
    df2["Kinase"] = "n/a"
    df2["Group"]= "n/a"
    dfy = df2[["SMILES", "Class", "Value", "Kinase", "Group"]]
    del df2
    
    return dfy
    
def validate_df(df):
    required_cols = {"SMILES", "Class", "Value", "Kinase", "Group", "Fingerprint"}
    if df.shape[0] <= 1000:
        raise ValueError(f"Not enough data: {df.shape[0]} elements")
    if set(df.columns) != required_cols:
        raise TypeError(f"Incorrect dataframe structure: \nProvided {list(df.columns)} \nExpected {list(required_cols)}")

def splits(df, mode= 1):
    if mode == 1:
        y = df["Class"]
    elif mode == 0:
        y = df["Value"]
    else:
        raise ValueError(f"{mode} is not a valid key. Use mode= 1 (clf) or mode= 0 (reg)")

    X = df["Fingerprint"]
    X_remainder, X_test, y_remainder, y_test = train_test_split(X, y, test_size= 0.2)
    X_train, X_val, y_train, y_val = train_test_split(X_remainder, y_remainder, test_size= 0.25)

    return X_train, X_test, X_val, y_train, y_test, y_val

def gen_fps(df, fingerprint_type="Morgan", resolution=1024):
    if "SMILES" not in df.columns:
        raise ValueError("DataFrame must contain a 'SMILES' column.")
    
    smiles = df["SMILES"].tolist()
    mols = [Chem.MolFromSmiles(s) for s in smiles if Chem.MolFromSmiles(s)]
    del smiles
    if fingerprint_type == 'Morgan':
        generator = rdFingerprintGenerator.GetMorganGenerator(radius=3, fpSize=resolution)
    elif fingerprint_type == 'AtomPair':
        generator = rdFingerprintGenerator.GetAtomPairGenerator(fpSize=resolution)
    else:
        raise ValueError(f"Unsupported fingerprint type: {fingerprint_type}")
    fingerprints = generator.GetFingerprints(mols)
    del mols

    return fingerprints
    del fingerprints

def hyper_params(clf, params, scoring, X, y):
    grid = GridSearchCV(clf, params, cv= 5, scoring= scoring)
    grid.fit(X, y)
    cv_results = grid.cv_results_
    all_scores = cv_results['mean_test_score']
    best_index = grid.best_index_
    fold_scores = [
        cv_results['split0_test_score'][best_index], cv_results['split1_test_score'][best_index], cv_results['split2_test_score'][best_index], cv_results['split3_test_score'][best_index], cv_results['split4_test_score'][best_index]
    ]
    
    return grid.best_params_, grid.best_score_, fold_scores
    del grid

def run_friedman_nemenyi(metric_matrix, metric_name):
    print(f"\nTesting: {metric_name}")
    stat, p = friedmanchisquare(*[metric_matrix[:, i] for i in range(metric_matrix.shape[1])])
    print(f"Friedman test statistic: {stat:.4f}, p-value: {p:.4f}")
    if p < 0.05:
        print(f"Significant differences found. \n\nNemenyi post-hoc test for {metric_name}:")
        df = pd.DataFrame(metric_matrix, columns=models)
        nemenyi = sp.posthoc_nemenyi_friedman(df)
        print(nemenyi)
    else:
        print("No significant differences found.")

def create_ensemble_model(models_dict):
    estimators = [(name, clf) for name, clf in models_dict.items()]
    ensemble_clf = VotingClassifier(
        estimators=estimators,
        voting="soft"
    )
    
    return ensemble_clf

In [5]:
# define lists, dicts, dataframes etc.

# Light Gradient Boosting (LGB)
clfLGB = LGBMClassifier(random_state = 42)
regLGB = LGBMRegressor(random_state = 42)
LGB_params = {
    "num_leaves": [10, 50, 100],
    "max_depth": [-1, 2, 5],
    "learning_rate": [0.01, 0.1, 0.2],
    "feature_fraction": [0.1, 0.5, 1]
}
# 3x3x3x3 training grid - 81 runs

# Extreme Gradient Boosting (XGB)
clfXGB = XGBClassifier(random_state = 42)
regXGB = XGBRegressor(random_state = 42)
XGB_params = {
    "n_estimators": [50, 100, 200],
    "max_depth": [-1, 2, 5],
    "learning_rate": [0.01, 0.1, 0.2],
    "subsample": [0.1, 0.5, 1.0]
}
# 3x3x3x3 training grid - 81 runs

# Multilayer Perceptron (MLP)
clfMLP = MLPClassifier(random_state = 42)
regMLP = MLPRegressor(random_state = 42)
MLP_params = {
    "hidden_layer_sizes": [(100,), (10,10)],
    "learning_rate_init": [0.001, 0.002, 0.005],
    "alpha": [0.0001, 0.0002]
}
# 2x3x2 training grid - 12 runs

# Random Forest (RF)
clfRF = RandomForestClassifier(random_state = 42)
regRF = RandomForestRegressor(random_state = 42)
RF_params = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 2, 5],
    "max_features": [10, 100],
    "min_samples_split": [2, 5, 7]
}
# 3x3x3x3 training grid - 81 runs

# Support Vector Machine (SVM)
clfSVM = SVC()
regSVM = SVR()
SVM_params = {
    "C": [1, 0.5, 2, 5],
    "kernel": ["linear", "rbf"]
}
# 4x2 training grid - 8 runs

# CatBoost (CB)
clfCB = CatBoostClassifier(random_seed = 42)
regCB = CatBoostRegressor(random_seed = 42)
CB_params = {
    "iterations": [500],
    "depth": [2, 5, 7],
    "learning_rate": [0.01, 0.02, 0.03],
    "l2_leaf_reg": [1.0, 2.0, 3.0]
}
# 3x3x3x3 training grid - 81 runs

classifiers = {
    "LGB": LGBMClassifier(random_state= 42, **LGB_params),
    "XGB": XGBClassifier(random_state= 42, **XGB_params),
    "MLP": MLPClassifier(random_state= 42, **MLP_params),
    "RF": RandomForestClassifier(random_state= 42, **RF_params),
    "SVC": SVC(**SVM_params),
    "CB": CatBoostClassifier(random_seed= 42, **CB_params)
}

regressors = {
    "LGB": LGBMRegressor(random_state= 42, **LGB_params),
    "XGB": XGBRegressor(random_state= 42, **XGB_params),
    "MLP": MLPRegressor(random_state= 42, **MLP_params),
    "RF": RandomForestRegressor(random_state= 42, **RF_params),
    "SVC": SVR(**SVM_params),
    "CB": CatBoostRegressor(random_seed= 42, **CB_params)
}

models=  ["LGB", "XGB", "MLP", "RF", "SVM", "CatBoost"]


In [6]:
df_KDR = df1[df1["Kinase_name"] == "KDR"]
df_FLT1 = df1[df1["Kinase_name"] == "FLT1"]
df_p110a = df1[df1["Kinase_name"] == "p110a"]
df_JAK1 = df1[df1["Kinase_name"] == "JAK1"]
df_JAK2 = df1[df1["Kinase_name"] == "JAK2"]
df_ErbB2 = df1[df1["Kinase_name"] == "ErbB2"]
df_EGFR = df1[df1["Kinase_name"] == "EGFR"]
df_PIM1 = df1[df1["Kinase_name"] == "PIM1"]
df_ROCK1 = df1[df1["Kinase_name"] == "ROCK1"]
df_ABL1 = df1[df1["Kinase_name"] == "ABL1"]

df_TK = df1[df1["Kinase_group"] == "TK"]
df_CMGC = df1[df1["Kinase_group"] == "CMGC"]
df_AGC = df1[df1["Kinase_group"] == "AGC"]
df_CAMK = df1[df1["Kinase_group"] == "CAMK"]
df_Atypical = df1[df1["Kinase_group"] == "Atypical"]

kinase_dfs = [
    df_KDR,
    df_FLT1,
    df_p110a,
    df_JAK1,
    df_JAK2,
    df_ErbB2,
    df_EGFR,
    df_PIM1,
    df_ROCK1,
    df_ABL1,
    df_TK,
    df_CMGC,
    df_AGC,
    df_CAMK,
    df_Atypical
]
cleaned_kinase_dfs1 = [clean_df1(i) for i in kinase_dfs]
del kinase_dfs

df2 = clean_df2(df2)

cleaned_kinase_dfs2 = []
z = 1
for i in cleaned_kinase_dfs1:
    i["Fingerprint"] = gen_fps(i, "Morgan", 2048)
    cleaned_kinase_dfs2.append(i)
    print("Iteration ",z," Complete") # for my sanity, waste of computation
    z+=1
del cleaned_kinase_dfs1

# Necessary to stop jupyter from breaking (too much data per package -> break into smaller packages)
dfs = np.array_split(df2, 10)
cleaned_dfs = []
for i in dfs:
    i["Fingerprint"] = gen_fps(i, "Morgan", 2048)
    cleaned_dfs.append(i)
    print("Iteration ",z," Complete")
    z+=1
del dfs
df2 = pd.concat([i for i in cleaned_dfs], axis= 0)
del cleaned_dfs, z

for i in cleaned_kinase_dfs2:
    validate_df(i)
validate_df(df2)

dataframes = [pd.concat([i, df2], axis= 0, ignore_index= True) for i in cleaned_kinase_dfs2]
del df2, cleaned_kinase_dfs2

X= 1
df_KDR = dataframes[0] # Reassign to dataframe
X_train_KDR_class, X_test_KDR_class, X_val_KDR_class, y_train_KDR_class, y_test_KDR_class, y_val_KDR_class = splits(df_KDR, mode= 1) # Classification
X_train_KDR_reg, X_test_KDR_reg, X_val_KDR_reg, y_train_KDR_reg, y_test_KDR_reg, y_val_KDR_reg = splits(df_KDR, mode= 0) # Regression
print("Split ", X, " Complete")
X +=1
df_FLT1 = dataframes[1]
X_train_FLT1_class, X_test_FLT1_class, X_val_FLT1_class, y_train_FLT1_class, y_test_FLT1_class, y_val_FLT1_class = splits(df_KDR, mode= 1)
X_train_FLT1_reg, X_test_FLT1_reg, X_val_FLT1_reg, y_train_FLT1_reg, y_test_FLT1_reg, y_val_FLT1_reg = splits(df_KDR, mode= 0)
print("Split ", X, " Complete")
X +=1
df_p110a = dataframes[2]
X_train_p110a_class, X_test_p110a_class, X_val_p110a_class, y_train_p110a_class, y_test_p110a_class, y_val_p110a_class = splits(df_KDR, mode= 1)
X_train_p110a_reg, X_test_p110a_reg, X_val_p110a_reg, y_train_p110a_reg, y_test_p110a_reg, y_val_p110a_reg = splits(df_KDR, mode= 0)
print("Split ", X, " Complete")
X +=1
df_JAK1 = dataframes[3]
X_train_JAK1_class, X_test_JAK1_class, X_val_JAK1_class, y_train_JAK1_class, y_test_JAK1_class, y_val_JAK1_class = splits(df_KDR, mode= 1)
X_train_JAK1_reg, X_test_JAK1_reg, X_val_JAK1_reg, y_train_JAK1_reg, y_test_JAK1_reg, y_val_JAK1_reg = splits(df_KDR, mode= 0)
print("Split ", X, " Complete")
X +=1
df_JAK2 = dataframes[4]
X_train_JAK2_class, X_test_JAK2_class, X_val_JAK2_class, y_train_JAK2_class, y_test_JAK2_class, y_val_JAK2_class = splits(df_KDR, mode= 1)
X_train_JAK2_reg, X_test_JAK2_reg, X_val_JAK2_reg, y_train_JAK2_reg, y_test_JAK2_reg, y_val_JAK2_reg = splits(df_KDR, mode= 0)
print("Split ", X, " Complete")
X +=1
df_ErbB2 = dataframes[5]
X_train_ErbB2_class, X_test_ErbB2_class, X_val_ErbB2_class, y_train_ErbB2_class, y_test_ErbB2_class, y_val_ErbB2_class = splits(df_KDR, mode= 1)
X_train_ErbB2_reg, X_test_ErbB2_reg, X_val_ErbB2_reg, y_train_ErbB2_reg, y_test_ErbB2_reg, y_val_ErbB2_reg = splits(df_KDR, mode= 0)
print("Split ", X, " Complete")
X +=1
df_EGFR = dataframes[6]
X_train_EGFR_class, X_test_EGFR_class, X_val_EGFR_class, y_train_EGFR_class, y_test_EGFR_class, y_val_EGFR_class = splits(df_KDR, mode= 1)
X_train_EGFR_reg, X_test_EGFR_reg, X_val_EGFR_reg, y_train_EGFR_reg, y_test_EGFR_reg, y_val_EGFR_reg = splits(df_KDR, mode= 0)
print("Split ", X, " Complete")
X +=1
df_PIM1 = dataframes[7]
X_train_PIM1_class, X_test_PIM1_class, X_val_PIM1_class, y_train_PIM1_class, y_test_PIM1_class, y_val_PIM1_class = splits(df_KDR, mode= 1)
X_train_PIM1_reg, X_test_PIM1_reg, X_val_PIM1_reg, y_train_PIM1_reg, y_test_PIM1_reg, y_val_PIM1_reg = splits(df_KDR, mode= 0)
print("Split ", X, " Complete")
X +=1
df_ROCK1 = dataframes[8]
X_train_ROCK1_class, X_test_ROCK1_class, X_val_ROCK1_class, y_train_ROCK1_class, y_test_ROCK1_class, y_val_ROCK1_class = splits(df_KDR, mode= 1)
X_train_ROCK1_reg, X_test_ROCK1_reg, X_val_ROCK1_reg, y_train_ROCK1_reg, y_test_ROCK1_reg, y_val_ROCK1_reg = splits(df_KDR, mode= 0)
print("Split ", X, " Complete")
X +=1
df_ABL1 = dataframes[9]
X_train_ABL1_class, X_test_ABL1_class, X_val_ABL1_class, y_train_ABL1_class, y_test_ABL1_class, y_val_ABL1_class = splits(df_KDR, mode= 1)
X_train_ABL1_reg, X_test_ABL1_reg, X_val_ABL1_reg, y_train_ABL1_reg, y_test_ABL1_reg, y_val_ABL1_reg = splits(df_KDR, mode= 0)
print("Split ", X, " Complete")
X +=1
df_TK = dataframes[10]
X_train_TK_class, X_test_TK_class, X_val_TK_class, y_train_TK_class, y_test_TK_class, y_val_TK_class = splits(df_KDR, mode= 1)
X_train_TK_reg, X_test_TK_reg, X_val_TK_reg, y_train_TK_reg, y_test_TK_reg, y_val_TK_reg = splits(df_KDR, mode= 0)
print("Split ", X, " Complete")
X +=1
df_CMGC = dataframes[11]
X_train_CMGC_class, X_test_CMGC_class, X_val_CMGC_class, y_train_CMGC_class, y_test_CMGC_class, y_val_CMGC_class = splits(df_KDR, mode= 1)
X_train_CMGC_reg, X_test_CMGC_reg, X_val_CMGC_reg, y_train_CMGC_reg, y_test_CMGC_reg, y_val_CMGC_reg = splits(df_KDR, mode= 0)
print("Split ", X, " Complete")
X +=1
df_AGC = dataframes[12]
X_train_AGC_class, X_test_AGC_class, X_val_AGC_class, y_train_AGC_class, y_test_AGC_class, y_val_AGC_class = splits(df_KDR, mode= 1)
X_train_AGC_reg, X_test_AGC_reg, X_val_AGC_reg, y_train_AGC_reg, y_test_AGC_reg, y_val_AGC_reg = splits(df_KDR, mode= 0)
print("Split ", X, " Complete")
X +=1
df_CAMK = dataframes[13]
X_train_CAMK_class, X_test_CAMK_class, X_val_CAMK_class, y_train_CAMK_class, y_test_CAMK_class, y_val_CAMK_class = splits(df_KDR, mode= 1)
X_train_CAMK_reg, X_test_CAMK_reg, X_val_CAMK_reg, y_train_CAMK_reg, y_test_CAMK_reg, y_val_CAMK_reg = splits(df_KDR, mode= 0)
print("Split ", X, " Complete")
X +=1
df_Atypical = dataframes[14]
X_train_Atypical_class, X_test_Atypical_class, X_val_Atypical_class, y_train_Atypical_class, y_test_Atypical_class, y_val_Atypical_class = splits(df_KDR, mode= 1)
X_train_Atypical_reg, X_test_Atypical_reg, X_val_Atypical_reg, y_train_Atypical_reg, y_test_Atypical_reg, y_val_Atypical_reg = splits(df_KDR, mode= 0)
print("Split ", X, " Complete")
del X

Iteration  1  Complete
Iteration  2  Complete
Iteration  3  Complete
Iteration  4  Complete
Iteration  5  Complete
Iteration  6  Complete
Iteration  7  Complete
Iteration  8  Complete
Iteration  9  Complete
Iteration  10  Complete
Iteration  11  Complete
Iteration  12  Complete
Iteration  13  Complete
Iteration  14  Complete
Iteration  15  Complete
Iteration  16  Complete
Iteration  17  Complete
Iteration  18  Complete
Iteration  19  Complete
Iteration  20  Complete
Iteration  21  Complete
Iteration  22  Complete
Iteration  23  Complete
Iteration  24  Complete
Iteration  25  Complete
Split  1  Complete
Split  2  Complete
Split  3  Complete
Split  4  Complete
Split  5  Complete
Split  6  Complete
Split  7  Complete
Split  8  Complete
Split  9  Complete
Split  10  Complete
Split  11  Complete
Split  12  Complete
Split  13  Complete
Split  14  Complete
Split  15  Complete


In [ ]:
# Testing for Speed

# RandomForestClassifier(n_jobs = -1) -> use all available cores. default is set to n_jobs = None -> one core (no parallel processing)
# Works for XGB, LGB and (conditionally) SVM. CatBoost uses thread_count = -1 instead. Not supported on MLP
# On this system (AI7 350 & 16gb ram), (should) increase speed by ~8x
# Note: do not run on battery for this, automatically parks 12/16 threads (uses 4 single-threaded cores) <- in the case of my laptop

clfRF = RandomForestClassifier(random_state = 42, n_jobs= -1)
regRF = RandomForestRegressor(random_state = 42, n_jobs= -1)
RF_params = {
    "n_estimators": [50, 100],
    "max_depth": [None],
    "max_features": [10],
    "min_samples_split": [2]
}
best_params, best_score, best_scores = hyper_params(clfSVM, SVM_params, "accuracy", np.stack(X_val_ABL1_class), y_val_ABL1_class,)
#^runs in ~55 mins (running 8 param combos in 2-fold, wtf??????)
#running for literally one change in parameters (50/100 estimators): start 19.29, end (cancelled) 20.29

*Do Not Run Cell Above*


It takes about half a decade to run

In [7]:
# Setting up classifiers / regressors using default params (estimated ~2.5-12.5 year runtime for parameter tuning on this device, we would need a server)
"""
# Light Gradient Boosting (LGB)
clfLGB = LGBMClassifier(random_state = 42)
regLGB = LGBMRegressor(random_state = 42)

# Extreme Gradient Boosting (XGB)
clfXGB = XGBClassifier(random_state = 42)
regXGB = XGBRegressor(random_state = 42)

# Multilayer Perceptron (MLP)
clfMLP = MLPClassifier(random_state = 42)
regMLP = MLPRegressor(random_state = 42)

# Random Forest (RF)
clfRF = RandomForestClassifier(random_state = 42)
regRF = RandomForestRegressor(random_state = 42)

# Support Vector Machine (SVM)
clfSVM = SVC()
regSVM = SVR()

# CatBoost (CB)
clfCB = CatBoostClassifier(random_seed = 42)
regCB = CatBoostRegressor(random_seed = 42)
"""
classifiers = {
    "LGB": LGBMClassifier(random_state= 42),
    "XGB": XGBClassifier(random_state= 42, objective='binary:logistic'),
    "MLP": MLPClassifier(random_state= 42),
    "RF": RandomForestClassifier(random_state= 42),
    "SVC": SVC(probability= True),
    "CB": CatBoostClassifier(random_seed= 42, verbose= False)
}

regressors = {
    "LGB": LGBMRegressor(random_state= 42),
    "XGB": XGBRegressor(random_state= 42),
    "MLP": MLPRegressor(random_state= 42),
    "RF": RandomForestRegressor(random_state= 42),
    "SVC": SVR(),
    "CB": CatBoostRegressor(random_seed= 42)
}

models=  ["LGB", "XGB", "MLP", "RF", "SVM", "CatBoost"]


In [8]:
from sklearn.svm import LinearSVC
classifiers = {
    "LGB": LGBMClassifier(random_state= 42),
    "MLP": MLPClassifier(random_state= 42),
    "RF": RandomForestClassifier(random_state= 42),
    "SVC": LinearSVC(),
    "CB": CatBoostClassifier(random_seed= 42, verbose= False)
}

def create_ensemble_model(models_dict):
    estimators = [(name, clf) for name, clf in models_dict.items()]
    ensemble_clf = VotingClassifier(
        estimators=estimators,
        voting="soft"
    )
    return ensemble_clf

clf = create_ensemble_model(classifiers)

clf.fit(np.stack(X_train_EGFR_class), y_train_EGFR_class)
# 15 minute training time roughly? (didn't keep track)

[LightGBM] [Info] Number of positive: 3292, number of negative: 470427
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.715982 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4096
[LightGBM] [Info] Number of data points in the train set: 473719, number of used features: 2048
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.006949 -> initscore=-4.962146
[LightGBM] [Info] Start training from score -4.962146


,estimators,"[('LGB', ...), ('MLP', ...), ...]"
,voting,'soft'
,weights,None
,n_jobs,None
,flatten_transform,True
,verbose,False
,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100


In [18]:

from sklearn.metrics import accuracy_score
y_pred_EGFR_class = clf.predict(np.stack(X_test_EGFR_class))
accuracy = accuracy_score(y_test_EGFR_class, y_pred_EGFR_class)
print("Accuracy:", accuracy)


AttributeError: 'LinearSVC' object has no attribute 'predict_proba'

In [21]:
scores = clf.decision_function(X_test_EGFR_class)

AttributeError: 'VotingClassifier' object has no attribute 'decision_function'

In [26]:
from sklearn.svm import LinearSVC
# 15 minute training time roughly? (didn't keep track)
# Calibrated classifier lets svc work with ensemble voting
from sklearn.calibration import CalibratedClassifierCV
svc_calibrated = CalibratedClassifierCV(SVC, method='sigmoid')
classifiers = {
    "LGB": LGBMClassifier(random_state= 42),
    "MLP": MLPClassifier(random_state= 42),
    "RF": RandomForestClassifier(random_state= 42),
    "SVC": svc_calibrated,
    "CB": CatBoostClassifier(random_seed= 42, verbose= False)
}

def create_ensemble_model(models_dict):
    estimators = [(name, clf) for name, clf in models_dict.items()]
    ensemble_clf = VotingClassifier(
        estimators=estimators,
        voting="soft"
    )
    return ensemble_clf

clf = create_ensemble_model(classifiers)

clf.fit(np.stack(X_train_EGFR_class), y_train_EGFR_class)

TypeError: BaseSVC.__sklearn_tags__() missing 1 required positional argument: 'self'

In [ ]:
for i in classifiers:
    i.fit(np.stack(X_train_EGFR_class), y_train_EGFR_class)

for i in classifiers, clf:
    accuracy = i.score(np.stack(X_test_EGFR_class), y_test_EGFR_class)
    print(f"Accuracy: {accuracy:.4f}")


In [12]:
# Attempting to 
from scipy import sparse

ensemble = VotingClassifier(estimators=[
    ('SVC', SVC(probability=True)),
    ('RF', RandomForestClassifier(random_state= 42, n_jobs= -1)), 
    ('LGB', LGBMClassifier(random_state= 42, n_jobs= -1)),
    ('MLP', MLPClassifier(random_state= 42)),
    ('CB', CatBoostClassifier(random_seed= 42, verbose= False))
], voting='soft')

ensemble.fit(X_train_EGFR_class, y_train_EGFR_class)

ValueError: setting an array element with a sequence.

In [13]:
X_train_EGFR_class = np.stack(X_train_EGFR_class)
X_train_EGFR_class.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [17]:
np.savetxt("test.csv", X_train_EGFR_class, delimiter=",", fmt="%d")

In [ ]:
# Printing to dataframe -> will convert each train/test set to a dataframe with stacked molecular fingerprints, then print them to a .csv file called "variable name".csv

# Put them in a dictionary with their names as keys
dataframes = {
    'df1': df1,
    'df2': df2
}

# Save each to CSV using its name
for name, df in dataframes.items():
    df = np.stack(df)
    df.to_csv(f"{name}.csv", index=False)


In [18]:
printing_dict = {
    "X_train_KDR_class": X_train_KDR_class, 
    "X_test_KDR_class": X_test_KDR_class, 
    "X_val_KDR_class": X_val_KDR_class,
    "X_train_KDR_reg": X_train_KDR_reg, 
    "X_test_KDR_reg": X_test_KDR_reg, 
    "X_val_KDR_reg": X_val_KDR_reg, 
    "X_train_FLT1_class": X_train_FLT1_class, 
    "X_test_FLT1_class": X_test_FLT1_class, 
    "X_val_FLT1_class": X_val_FLT1_class, 
    "X_train_FLT1_reg": X_train_FLT1_reg, 
    "X_test_FLT1_reg": X_test_FLT1_reg, 
    "X_val_FLT1_reg": X_val_FLT1_reg, 
    "X_train_p110a_class": X_train_p110a_class, 
    "X_test_p110a_class": X_test_p110a_class, 
    "X_val_p110a_class": X_val_p110a_class, 
    "X_train_p110a_reg": X_train_p110a_reg, 
    "X_test_p110a_reg": X_test_p110a_reg, 
    "X_val_p110a_reg": X_val_p110a_reg, 
    "X_train_JAK1_class": X_train_JAK1_class, 
    "X_test_JAK1_class": X_test_JAK1_class, 
    "X_val_JAK1_class": X_val_JAK1_class, 
    "X_train_JAK1_reg": X_train_JAK1_reg, 
    "X_test_JAK1_reg": X_test_JAK1_reg, 
    "X_val_JAK1_reg": X_val_JAK1_reg, 
    "X_train_JAK2_class": X_train_JAK2_class, 
    "X_test_JAK2_class": X_test_JAK2_class, 
    "X_val_JAK2_class": X_val_JAK2_class, 
    "X_train_JAK2_reg": X_train_JAK2_reg, 
    "X_test_JAK2_reg": X_test_JAK2_reg, 
    "X_val_JAK2_reg": X_val_JAK2_reg, 
    "X_train_ErbB2_class": X_train_ErbB2_class, 
    "X_test_ErbB2_class": X_test_ErbB2_class, 
    "X_val_ErbB2_class": X_val_ErbB2_class, 
    "X_train_ErbB2_reg": X_train_ErbB2_reg, 
    "X_test_ErbB2_reg": X_test_ErbB2_reg, 
    "X_val_ErbB2_reg": X_val_ErbB2_reg, 
    "X_train_EGFR_class": X_train_EGFR_class, 
    "X_test_EGFR_class": X_test_EGFR_class, 
    "X_val_EGFR_class": X_val_EGFR_class, 
    "X_train_EGFR_reg": X_train_EGFR_reg, 
    "X_test_EGFR_reg": X_test_EGFR_reg, 
    "X_val_EGFR_reg": X_val_EGFR_reg, 
    "X_train_PIM1_class": X_train_PIM1_class, 
    "X_test_PIM1_class": X_test_PIM1_class, 
    "X_val_PIM1_class": X_val_PIM1_class, 
    "X_train_PIM1_reg": X_train_PIM1_reg, 
    "X_test_PIM1_reg": X_test_PIM1_reg, 
    "X_val_PIM1_reg": X_val_PIM1_reg, 
    "X_train_ROCK1_class": X_train_ROCK1_class, 
    "X_test_ROCK1_class": X_test_ROCK1_class, 
    "X_val_ROCK1_class": X_val_ROCK1_class, 
    "X_train_ROCK1_reg": X_train_ROCK1_reg, 
    "X_test_ROCK1_reg": X_test_ROCK1_reg, 
    "X_val_ROCK1_reg": X_val_ROCK1_reg, 
    "X_train_ABL1_class": X_train_ABL1_class, 
    "X_test_ABL1_class": X_test_ABL1_class, 
    "X_val_ABL1_class": X_val_ABL1_class, 
    "X_train_ABL1_reg": X_train_ABL1_reg, 
    "X_test_ABL1_reg": X_test_ABL1_reg, 
    "X_val_ABL1_reg": X_val_ABL1_reg, 
    "X_train_TK_class": X_train_TK_class, 
    "X_test_TK_class": X_test_TK_class, 
    "X_val_TK_class": X_val_TK_class, 
    "X_train_TK_reg": X_train_TK_reg, 
    "X_test_TK_reg": X_test_TK_reg, 
    "X_val_TK_reg": X_val_TK_reg, 
    "X_train_CMGC_class": X_train_CMGC_class, 
    "X_test_CMGC_class": X_test_CMGC_class, 
    "X_val_CMGC_class": X_val_CMGC_class, 
    "X_train_CMGC_reg": X_train_CMGC_reg, 
    "X_test_CMGC_reg": X_test_CMGC_reg, 
    "X_val_CMGC_reg": X_val_CMGC_reg, 
    "X_train_AGC_class": X_train_AGC_class, 
    "X_test_AGC_class": X_test_AGC_class, 
    "X_val_AGC_class": X_val_AGC_class, 
    "X_train_AGC_reg": X_train_AGC_reg, 
    "X_test_AGC_reg": X_test_AGC_reg, 
    "X_val_AGC_reg": X_val_AGC_reg, 
    "X_train_CAMK_class": X_train_CAMK_class, 
    "X_test_CAMK_class": X_test_CAMK_class, 
    "X_val_CAMK_class": X_val_CAMK_class, 
    "X_train_CAMK_reg": X_train_CAMK_reg, 
    "X_test_CAMK_reg": X_test_CAMK_reg, 
    "X_val_CAMK_reg": X_val_CAMK_reg, 
    "X_train_Atypical_class": X_train_Atypical_class, 
    "X_test_Atypical_class": X_test_Atypical_class, 
    "X_val_Atypical_class": X_val_Atypical_class, 
    "X_train_Atypical_reg": X_train_Atypical_reg, 
    "X_test_Atypical_reg": X_test_Atypical_reg, 
    "X_val_Atypical_reg": X_val_Atypical_reg
}

In [19]:
for name, df in printing_dict.items():
    arr = np.stack(df)
    del df
    print(f"{name} converted to np.arr")
    np.savetxt(f"{name}.csv", arr, delimiter=",", fmt="%d")
    del arr
    print(f"{name} printed to .csv file")

print("end")

X_train_KDR_class converted to np.arr
X_train_KDR_class printed to .csv file
X_test_KDR_class converted to np.arr
X_test_KDR_class printed to .csv file
X_val_KDR_class converted to np.arr
X_val_KDR_class printed to .csv file
X_train_KDR_reg converted to np.arr
X_train_KDR_reg printed to .csv file
X_test_KDR_reg converted to np.arr
X_test_KDR_reg printed to .csv file
X_val_KDR_reg converted to np.arr
X_val_KDR_reg printed to .csv file
X_train_FLT1_class converted to np.arr
X_train_FLT1_class printed to .csv file
X_test_FLT1_class converted to np.arr
X_test_FLT1_class printed to .csv file
X_val_FLT1_class converted to np.arr
X_val_FLT1_class printed to .csv file
X_train_FLT1_reg converted to np.arr
X_train_FLT1_reg printed to .csv file
X_test_FLT1_reg converted to np.arr
X_test_FLT1_reg printed to .csv file
X_val_FLT1_reg converted to np.arr
X_val_FLT1_reg printed to .csv file
X_train_p110a_class converted to np.arr
X_train_p110a_class printed to .csv file
X_test_p110a_class converted to

In [20]:
printing_dict_y = {
    "y_train_KDR_class": y_train_KDR_class, 
    "y_test_KDR_class": y_test_KDR_class, 
    "y_val_KDR_class": y_val_KDR_class,
    "y_train_KDR_reg": y_train_KDR_reg, 
    "y_test_KDR_reg": y_test_KDR_reg, 
    "y_val_KDR_reg": y_val_KDR_reg, 
    "y_train_FLT1_class": y_train_FLT1_class, 
    "y_test_FLT1_class": y_test_FLT1_class, 
    "y_val_FLT1_class": y_val_FLT1_class, 
    "y_train_FLT1_reg": y_train_FLT1_reg, 
    "y_test_FLT1_reg": y_test_FLT1_reg, 
    "y_val_FLT1_reg": y_val_FLT1_reg, 
    "y_train_p110a_class": y_train_p110a_class, 
    "y_test_p110a_class": y_test_p110a_class, 
    "y_val_p110a_class": y_val_p110a_class, 
    "y_train_p110a_reg": y_train_p110a_reg, 
    "y_test_p110a_reg": y_test_p110a_reg, 
    "y_val_p110a_reg": y_val_p110a_reg, 
    "y_train_JAK1_class": y_train_JAK1_class, 
    "y_test_JAK1_class": y_test_JAK1_class, 
    "y_val_JAK1_class": y_val_JAK1_class, 
    "y_train_JAK1_reg": y_train_JAK1_reg, 
    "y_test_JAK1_reg": y_test_JAK1_reg, 
    "y_val_JAK1_reg": y_val_JAK1_reg, 
    "y_train_JAK2_class": y_train_JAK2_class, 
    "y_test_JAK2_class": y_test_JAK2_class, 
    "y_val_JAK2_class": y_val_JAK2_class, 
    "y_train_JAK2_reg": y_train_JAK2_reg, 
    "y_test_JAK2_reg": y_test_JAK2_reg, 
    "y_val_JAK2_reg": y_val_JAK2_reg, 
    "y_train_ErbB2_class": y_train_ErbB2_class, 
    "y_test_ErbB2_class": y_test_ErbB2_class, 
    "y_val_ErbB2_class": y_val_ErbB2_class, 
    "y_train_ErbB2_reg": y_train_ErbB2_reg, 
    "y_test_ErbB2_reg": y_test_ErbB2_reg, 
    "y_val_ErbB2_reg": y_val_ErbB2_reg, 
    "y_train_EGFR_class": y_train_EGFR_class, 
    "y_test_EGFR_class": y_test_EGFR_class, 
    "y_val_EGFR_class": y_val_EGFR_class, 
    "y_train_EGFR_reg": y_train_EGFR_reg, 
    "y_test_EGFR_reg": y_test_EGFR_reg, 
    "y_val_EGFR_reg": y_val_EGFR_reg, 
    "y_train_PIM1_class": y_train_PIM1_class, 
    "y_test_PIM1_class": y_test_PIM1_class, 
    "y_val_PIM1_class": y_val_PIM1_class, 
    "y_train_PIM1_reg": y_train_PIM1_reg, 
    "y_test_PIM1_reg": y_test_PIM1_reg, 
    "y_val_PIM1_reg": y_val_PIM1_reg, 
    "y_train_ROCK1_class": y_train_ROCK1_class, 
    "y_test_ROCK1_class": y_test_ROCK1_class, 
    "y_val_ROCK1_class": y_val_ROCK1_class, 
    "y_train_ROCK1_reg": y_train_ROCK1_reg, 
    "y_test_ROCK1_reg": y_test_ROCK1_reg, 
    "y_val_ROCK1_reg": y_val_ROCK1_reg, 
    "y_train_ABL1_class": y_train_ABL1_class, 
    "y_test_ABL1_class": y_test_ABL1_class, 
    "y_val_ABL1_class": y_val_ABL1_class, 
    "y_train_ABL1_reg": y_train_ABL1_reg, 
    "y_test_ABL1_reg": y_test_ABL1_reg, 
    "y_val_ABL1_reg": y_val_ABL1_reg, 
    "y_train_TK_class": y_train_TK_class, 
    "y_test_TK_class": y_test_TK_class, 
    "y_val_TK_class": y_val_TK_class, 
    "y_train_TK_reg": y_train_TK_reg, 
    "y_test_TK_reg": y_test_TK_reg, 
    "y_val_TK_reg": y_val_TK_reg, 
    "y_train_CMGC_class": y_train_CMGC_class, 
    "y_test_CMGC_class": y_test_CMGC_class, 
    "y_val_CMGC_class": y_val_CMGC_class, 
    "y_train_CMGC_reg": y_train_CMGC_reg, 
    "y_test_CMGC_reg": y_test_CMGC_reg, 
    "y_val_CMGC_reg": y_val_CMGC_reg, 
    "y_train_AGC_class": y_train_AGC_class, 
    "y_test_AGC_class": y_test_AGC_class, 
    "y_val_AGC_class": y_val_AGC_class, 
    "y_train_AGC_reg": y_train_AGC_reg, 
    "y_test_AGC_reg": y_test_AGC_reg, 
    "y_val_AGC_reg": y_val_AGC_reg, 
    "y_train_CAMK_class": y_train_CAMK_class, 
    "y_test_CAMK_class": y_test_CAMK_class, 
    "y_val_CAMK_class": y_val_CAMK_class, 
    "y_train_CAMK_reg": y_train_CAMK_reg, 
    "y_test_CAMK_reg": y_test_CAMK_reg, 
    "y_val_CAMK_reg": y_val_CAMK_reg, 
    "y_train_Atypical_class": y_train_Atypical_class, 
    "y_test_Atypical_class": y_test_Atypical_class, 
    "y_val_Atypical_class": y_val_Atypical_class, 
    "y_train_Atypical_reg": y_train_Atypical_reg, 
    "y_test_Atypical_reg": y_test_Atypical_reg, 
    "y_val_Atypical_reg": y_val_Atypical_reg
}

In [21]:
for name, df in printing_dict_y.items():
    df.to_csv(f"{name}.csv")
    del df
    print(f"{name} printed to .csv file")

print("end")

y_train_KDR_class printed to .csv file
y_test_KDR_class printed to .csv file
y_val_KDR_class printed to .csv file
y_train_KDR_reg printed to .csv file
y_test_KDR_reg printed to .csv file
y_val_KDR_reg printed to .csv file
y_train_FLT1_class printed to .csv file
y_test_FLT1_class printed to .csv file
y_val_FLT1_class printed to .csv file
y_train_FLT1_reg printed to .csv file
y_test_FLT1_reg printed to .csv file
y_val_FLT1_reg printed to .csv file
y_train_p110a_class printed to .csv file
y_test_p110a_class printed to .csv file
y_val_p110a_class printed to .csv file
y_train_p110a_reg printed to .csv file
y_test_p110a_reg printed to .csv file
y_val_p110a_reg printed to .csv file
y_train_JAK1_class printed to .csv file
y_test_JAK1_class printed to .csv file
y_val_JAK1_class printed to .csv file
y_train_JAK1_reg printed to .csv file
y_test_JAK1_reg printed to .csv file
y_val_JAK1_reg printed to .csv file
y_train_JAK2_class printed to .csv file
y_test_JAK2_class printed to .csv file
y_val_JAK